# Clean raw data 

This notebook reads data from [../data/raw](../data/raw), fixes up known issues and then writes the cleaned data to [../data/cleaned](../data/cleaned).

See cells below for the various data fixes applied.

In [1]:
# All imports live here.

import pandas as pd
pd.options.plotting.backend = 'plotly'

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from datetime import timedelta

In [2]:
# Load raw data
with open('../data/raw/playwright.json') as f:
    tests = pd.read_json(f)
print('Loaded tests with %d rows. Columns:' % (tests.shape[0],))
print(tests.dtypes)

with open('../data/raw/jobs.json') as f:
    jobs = pd.read_json(f)
print('Loaded jobs with %d rows. Columns:' % (jobs.shape[0],))
print(jobs.dtypes)

Loaded tests with 1138050 rows. Columns:
meta_id                                  int64
workflow_id                              int64
workflow_run_number                      int64
workflow_run_attempt                     int64
workflow_created_at        datetime64[ns, UTC]
workflow_event                          object
workflow_head_branch                    object
commit_sha                              object
build_flavor                            object
composite                               object
pw_suite_title                          object
pw_spec_title                           object
pw_test_project_name                    object
pw_test_expected_status                 object
pw_test_status                          object
pw_result_status                        object
pw_result_duration                       int64
pw_result_retry                          int64
dtype: object
Loaded jobs with 745999 rows. Columns:
workflow_id                           int64
workflow_run_att

## Remap project names

Projects were renamed by https://github.com/Azure/communication-ui-library/pull/2162


In [3]:
# Rename projects
PROJECT_NAMES = {
    'Desktop Chrome': 'desktop-chrome',
    'Mobile Android Portrait': 'mobile-android-portrait',
    'Mobile Android Landscape': 'mobile-android-landscape',
}

print('Raw data had {} unique projects'.format(len(tests['pw_test_project_name'].unique())))

def _project_name(row):
    n = row['pw_test_project_name']
    return PROJECT_NAMES.get(n, n)

tests['pw_test_project_name'] = tests.apply(_project_name, axis=1)
print('Cleaned data has {} unique projects'.format(len(tests['pw_test_project_name'].unique())))

Raw data had 6 unique projects
Cleaned data has 3 unique projects


# Augmented columns

Add derived columns to data.

In [4]:
# Augmented columns for tests
tests['workflow_created_date'] = tests['workflow_created_at'].dt.normalize()
tests['test_uid'] = tests['composite'].astype(str) + '/' + tests['pw_suite_title'].astype(str) + '/' + tests['pw_spec_title'].astype(str)
tests['pw_result_duration_seconds'] = tests['pw_result_duration'] / 1000

print('Augmented data with common derived columns:')
print(tests.dtypes)

Augmented data with common derived columns:
meta_id                                     int64
workflow_id                                 int64
workflow_run_number                         int64
workflow_run_attempt                        int64
workflow_created_at           datetime64[ns, UTC]
workflow_event                             object
workflow_head_branch                       object
commit_sha                                 object
build_flavor                               object
composite                                  object
pw_suite_title                             object
pw_spec_title                              object
pw_test_project_name                       object
pw_test_expected_status                    object
pw_test_status                             object
pw_result_status                           object
pw_result_duration                          int64
pw_result_retry                             int64
workflow_created_date         datetime64[ns, UTC]
test_u

In [5]:
# Augmented columns for jobs
jobs['job_started_date'] = jobs['job_started_at'].dt.normalize()
jobs['job_duration_minutes'] = (jobs['job_completed_at'] - jobs['job_started_at']).apply(lambda x: x.total_seconds() / 60.0)

jobs['workflow_attempt_uid'] = jobs['workflow_id'].astype(str) + '_' + jobs['workflow_run_attempt'].astype(str)

e2e_dimensions = {
    'Call Composite automation test (stable)': ('call', 'stable'),
    'Call Composite automation test (beta)': ('call', 'beta'),
    'Call With Chat Composite automation test (stable)': ('callWithChat', 'stable'),
    'Call With Chat Composite automation test (beta)': ('callWithChat', 'beta'),
    'Chat Composite automation test (stable)': ('chat', 'stable'),
    'Chat Composite automation test (beta)': ('chat', 'beta'),
}
jobs['e2e-composite'] = jobs['job_name'].map(lambda j: e2e_dimensions.get(j, (None, None))[0])
jobs['e2e-flavor'] = jobs['job_name'].map(lambda j: e2e_dimensions.get(j, (None, None))[1])

print('Augmented data with common derived columns:')
print(jobs.dtypes)

Augmented data with common derived columns:
workflow_id                           int64
workflow_run_attempt                  int64
job_status                           object
job_conclusion                       object
job_started_at          datetime64[ns, UTC]
job_completed_at        datetime64[ns, UTC]
job_name                             object
step_name                            object
step_status                          object
step_conclusion                      object
step_started_at                      object
step_completed_at                    object
job_started_date        datetime64[ns, UTC]
job_duration_minutes                float64
workflow_attempt_uid                 object
e2e-composite                        object
e2e-flavor                           object
dtype: object


## write cleaned data

In [6]:
print('Writing tests with %d rows. Columns:' % (tests.shape[0],))
print(tests.dtypes)
with open('../data/cleaned/playwright.json', 'w') as f:    
    tests.to_json(f, indent=1, date_format='iso')

print('Writing jobs with %d rows. Columns:' % (jobs.shape[0],))
print(jobs.dtypes)
with open('../data/cleaned/jobs.json', 'w') as f:
    jobs.to_json(f, indent=1, date_format='iso')

Writing tests with 1138050 rows. Columns:
meta_id                                     int64
workflow_id                                 int64
workflow_run_number                         int64
workflow_run_attempt                        int64
workflow_created_at           datetime64[ns, UTC]
workflow_event                             object
workflow_head_branch                       object
commit_sha                                 object
build_flavor                               object
composite                                  object
pw_suite_title                             object
pw_spec_title                              object
pw_test_project_name                       object
pw_test_expected_status                    object
pw_test_status                             object
pw_result_status                           object
pw_result_duration                          int64
pw_result_retry                             int64
workflow_created_date         datetime64[ns, UTC]
test_uid